# --- Day 15: Chiton --- 

https://adventofcode.com/2021/day/15

## Get Input Data

In [1]:
import numpy as np

In [2]:
def parse_data(filename):
    """Read in risk map data."""

    with open(f'../inputs/{filename}') as file:
        risk_map = [[int(x) for x in line.strip()] for line in file.readlines()]

    risk_map = np.matrix(risk_map)
    return risk_map

In [3]:
test_risk_map = parse_data('test_risk_map.txt')
test_risk_map

matrix([[1, 1, 6, 3, 7, 5, 1, 7, 4, 2],
        [1, 3, 8, 1, 3, 7, 3, 6, 7, 2],
        [2, 1, 3, 6, 5, 1, 1, 3, 2, 8],
        [3, 6, 9, 4, 9, 3, 1, 5, 6, 9],
        [7, 4, 6, 3, 4, 1, 7, 1, 1, 1],
        [1, 3, 1, 9, 1, 2, 8, 1, 3, 7],
        [1, 3, 5, 9, 9, 1, 2, 4, 2, 1],
        [3, 1, 2, 5, 4, 2, 1, 6, 3, 9],
        [1, 2, 9, 3, 1, 3, 8, 5, 2, 1],
        [2, 3, 1, 1, 9, 4, 4, 5, 8, 1]])

In [4]:
smaller_test_risk_map = test_risk_map[:3, :3]
smaller_test_risk_map

matrix([[1, 1, 6],
        [1, 3, 8],
        [2, 1, 3]])

In [5]:
risk_map = parse_data('risk_map.txt')
risk_map.shape

(100, 100)

## Part 1
---

Will need to apply [Dijkstra's Algorithm](https://en.wikipedia.org/wiki/Dijkstra%27s_algorithm) to solve.

Chapter 7 in [Grokking Algorithms](https://www.manning.com/books/grokking-algorithms) provides an excellent tutorial on the algorithm -- highly recommend this book:

[![Grokking Algorithms](https://images.manning.com/360/480/resize/book/3/0b325da-eb26-4e50-8a2a-46042c647083/Bhargava-Algorithms_hires.png)](https://www.manning.com/books/grokking-algorithms)

Code for the algorithm can be found [here](https://github.com/egonSchiele/grokking_algorithms/blob/master/07_dijkstras_algorithm/python/01_dijkstras_algorithm.py).

<br>

This [blog post](https://www.udacity.com/blog/2021/10/implementing-dijkstras-algorithm-in-python.html) from Udacity also gives an excellent overview/tutorial/code example of the algorithm.

<br>

But, actually, both of those implementations of Dijkstra's Algorithm are fairly inefficient and require checking all nodes for every node ($O(n^2)$?).  

<br>

This implementation uses a "[priority queue]()," and goes MUCH, MUCH faster: https://bradfieldcs.com/algos/graphs/dijkstras-algorithm/

In [6]:
def get_neighbors(y, x, max_dim):
    """Return a list of [(y, x)] neighbor coordinates."""

    neighbors = [(y+1, x), (y-1, x), (y, x+1), (y, x-1)]
    
    # Filter out neighbors who have gone over the edge!
    good_neighbors = list(filter(lambda n: 0 <= n[0] <= max_dim and 0 <= n[1] <= max_dim, neighbors))

    return good_neighbors

In [7]:
def make_graph(map):
    """Convert a map into a dictionary/graph with appropriate weight/risk values for each node."""

    graph = {}
    # processed = []

    max_dim = map.shape[0] - 1

    for y in range(map.shape[0]):
        for x in range(map.shape[1]):

            # processed.append((y, x))
            neighbors = get_neighbors(y, x, max_dim)

            graph[(y, x)] = {}
            for n in neighbors:
                # if n not in processed:
                graph[(y, x)][n] = map[n]

    # Set the "finish" node to have no neighbors
    graph[(map.shape[0]-1, map.shape[1]-1)] = {}

    return graph

In [8]:
smaller_test_risk_graph = make_graph(smaller_test_risk_map)
smaller_test_risk_graph

{(0, 0): {(1, 0): 1, (0, 1): 1},
 (0, 1): {(1, 1): 3, (0, 2): 6, (0, 0): 1},
 (0, 2): {(1, 2): 8, (0, 1): 1},
 (1, 0): {(2, 0): 2, (0, 0): 1, (1, 1): 3},
 (1, 1): {(2, 1): 1, (0, 1): 1, (1, 2): 8, (1, 0): 1},
 (1, 2): {(2, 2): 3, (0, 2): 6, (1, 1): 3},
 (2, 0): {(1, 0): 1, (2, 1): 1},
 (2, 1): {(1, 1): 3, (2, 2): 3, (2, 0): 2},
 (2, 2): {}}

In [9]:
def make_risk_table(map):
    """Return a hash table for every node on the map. Set the risk values to infinity, except
    for the first two neighbor nodes."""

    risk_table = {}

    for y in range(map.shape[0]):
        for x in range(map.shape[1]):
            risk_table[(y, x)] = float('inf')

    # Get risk values for starting node
    max_dim = map.shape[0] - 1
    start_neighbors = get_neighbors(0, 0, max_dim)
    for n in start_neighbors:
        risk_table[n] = map[n]

    # Set starting node risk to 0
    risk_table[(0, 0)] = 0

    return risk_table

In [10]:
smaller_test_risk_table = make_risk_table(smaller_test_risk_map)
smaller_test_risk_table

{(0, 0): 0,
 (0, 1): 1,
 (0, 2): inf,
 (1, 0): 1,
 (1, 1): inf,
 (1, 2): inf,
 (2, 0): inf,
 (2, 1): inf,
 (2, 2): inf}

In [11]:
def find_lowest_risk_node(risk_table, processed_nodes):
    """Reten the node in the risk table with the lowest risk."""
    lowest_risk = float("inf")
    lowest_risk_node = None
    
    # Go through each node.  # KP: This is super inefficient!
    for node in risk_table:
        risk = risk_table[node]
        
        # If it's the lowest cost so far and hasn't been processed yet...
        if risk < lowest_risk and node not in processed_nodes:
            # ... set it as the new lowest-cost node.
            lowest_risk = risk
            lowest_risk_node = node

    return lowest_risk_node

In [12]:
def find_min_cum_risk(risk_graph, risk_table):
    """Use Dijkstra's algorithm to find the miniumum cumulative risk through the graph."""
    
    processed_nodes = [(0, 0)]
    node = find_lowest_risk_node(risk_table, processed_nodes)

    while node is not None:

        risk = risk_table[node]

        # Go through all the neighbors of this node.
        neighbors = risk_graph[node]
        for n in neighbors.keys():
            new_risk = risk + neighbors[n]

            # If it's less risky to get to this neighbor by going through this node...
            if new_risk < risk_table[n]:
                # ... update the risk for this node.
                risk_table[n] = new_risk

        # Mark the node as processed.
        processed_nodes.append(node)
        # print(processed_nodes)

        # Find the next node to process, and loop.
        node = find_lowest_risk_node(risk_table, processed_nodes)

    end_node = max(risk_table.keys())

    return risk_table[end_node]

In [13]:
def find1(current_min_node, unvisited_nodes, risk_table):
    """Slightly different implementation of find_lowest_risk_node().
    From Udacity blog post on Dijkstra's algorithm.
    """

    for node in unvisited_nodes: # Iterate over the nodes
        if current_min_node == None:
            current_min_node = node
        elif risk_table[node] < risk_table[current_min_node]:
            current_min_node = node

    return current_min_node

In [14]:
def find2(risk_table, unvisited):
    """My attempt at a slightly more efficient version of find1()"""

    # Source: https://stackoverflow.com/questions/21584601/most-efficient-way-to-find-the-key-of-the-smallest-value-in-a-dictionary-from-a
    min_node = min(risk_table.keys() & unvisited, key=risk_table.get)

    return min_node

In [15]:
def find_min_cum_risk2(risk_graph, risk_table):
    """Test out the Udacity implementation of Dijkstra's algorithm"""

    unvisited_nodes = list(risk_table.keys())

    while unvisited_nodes:

        # The code block below finds the node with the lowest score
        current_min_node = None

        # KP: This seems very inefficient!
        # current_min_node = find1(current_min_node, unvisited_nodes, risk_table)
        # KP: Test a (hopefully more efficient) approach
        current_min_node = find2(risk_table, unvisited_nodes)
        # KP: Turn out, they're not terribly different. Ugh.

        current_risk = risk_table[current_min_node]

        # Go through all the neighbors of this node.
        neighbors = risk_graph[current_min_node]
        for n in neighbors.keys():
            new_risk = current_risk + neighbors[n]

            # If it's less risky to get to this neighbor by going through this node...
            if new_risk < risk_table[n]:
                # ... update the risk for this node.
                risk_table[n] = new_risk

        unvisited_nodes.remove(current_min_node)

    end_node = max(risk_table.keys())

    return risk_table[end_node]

### Run on Test Data

In [16]:
%time find_min_cum_risk(smaller_test_risk_graph, smaller_test_risk_table)  # Should return 7

Wall time: 0 ns


7

In [17]:
%time find_min_cum_risk2(smaller_test_risk_graph, smaller_test_risk_table)  # Should return 7

Wall time: 0 ns


7

In [18]:
test_risk_table = make_risk_table(test_risk_map)
test_risk_graph = make_graph(test_risk_map)
%time find_min_cum_risk(test_risk_graph, test_risk_table)  # Should return 40

Wall time: 22 ms


40

In [19]:
%time find_min_cum_risk2(test_risk_graph, test_risk_table)  # Should return 40

Wall time: 2 ms


40

### Run on Input Data

In [20]:
risk_table = make_risk_table(risk_map)
risk_graph = make_graph(risk_map)
%time find_min_cum_risk2(risk_graph, risk_table)

Wall time: 2min 26s


824

### This one is the bomb.

Source: https://bradfieldcs.com/algos/graphs/dijkstras-algorithm/

The algorithm uses python's [heapq library](https://docs.python.org/3/library/heapq.html), which implements a min heap as the default, even though  
a max heap is more cannnical. 

In [21]:
import heapq

In [22]:
def calculate_risks(graph, starting_node):
    """Implementation of Dijkstra's agorithm, using a min heap priority queue.
    Returns the cumulative risk for the path with the lowest cumulative risk through
    the graph.
    """
    risks = {node: float('infinity') for node in graph}
    risks[starting_node] = 0

    pq = [(0, starting_node)]  # Priqority Queue
    while len(pq) > 0:
        current_risk, current_node = heapq.heappop(pq)

        # Nodes can get added to the priority queue multiple times.
        # Only process a node the first time we remove it from the priority queue.
        # KP: This isn't necessary.
        if current_risk > risks[current_node]:
            continue

        for neighbor, next_risk in graph[current_node].items():
            risk = current_risk + next_risk

            # Only consider this new path if it's better than any path we've already found.
            if risk < risks[neighbor]:
                risks[neighbor] = risk
                heapq.heappush(pq, (risk, neighbor))

    end_node = max(graph)

    return risks[end_node]

In [23]:
%time calculate_risks(smaller_test_risk_graph, (0, 0))

Wall time: 0 ns


7

In [24]:
%time calculate_risks(test_risk_graph, (0, 0))

Wall time: 0 ns


40

In [25]:
%time calculate_risks(risk_graph, (0, 0))

Wall time: 80 ms


824

## Part 2
---

In [26]:
def make_bigger_map(map):
    """Make a copy of map that is 5x bigger, and risk values increase to the right and down.
    Risk values wrap from 9 back to 1.
    """

    map = map.copy()

    rows = []
    for j in range(5):
        start_map = np.where(map + j <=9, map + j, (map + j) % 9)

        row = []
        for i in range(5):
            i_map = np.where(start_map + i <= 9, start_map + i, (start_map + i) % 9)
            row.append(i_map)

        rows.append(np.concatenate(row, axis=1))

    new_map = np.concatenate(rows, axis=0)

    return new_map

In [27]:
make_bigger_map(smaller_test_risk_map)

array([[1, 1, 6, 2, 2, 7, 3, 3, 8, 4, 4, 9, 5, 5, 1],
       [1, 3, 8, 2, 4, 9, 3, 5, 1, 4, 6, 2, 5, 7, 3],
       [2, 1, 3, 3, 2, 4, 4, 3, 5, 5, 4, 6, 6, 5, 7],
       [2, 2, 7, 3, 3, 8, 4, 4, 9, 5, 5, 1, 6, 6, 2],
       [2, 4, 9, 3, 5, 1, 4, 6, 2, 5, 7, 3, 6, 8, 4],
       [3, 2, 4, 4, 3, 5, 5, 4, 6, 6, 5, 7, 7, 6, 8],
       [3, 3, 8, 4, 4, 9, 5, 5, 1, 6, 6, 2, 7, 7, 3],
       [3, 5, 1, 4, 6, 2, 5, 7, 3, 6, 8, 4, 7, 9, 5],
       [4, 3, 5, 5, 4, 6, 6, 5, 7, 7, 6, 8, 8, 7, 9],
       [4, 4, 9, 5, 5, 1, 6, 6, 2, 7, 7, 3, 8, 8, 4],
       [4, 6, 2, 5, 7, 3, 6, 8, 4, 7, 9, 5, 8, 1, 6],
       [5, 4, 6, 6, 5, 7, 7, 6, 8, 8, 7, 9, 9, 8, 1],
       [5, 5, 1, 6, 6, 2, 7, 7, 3, 8, 8, 4, 9, 9, 5],
       [5, 7, 3, 6, 8, 4, 7, 9, 5, 8, 1, 6, 9, 2, 7],
       [6, 5, 7, 7, 6, 8, 8, 7, 9, 9, 8, 1, 1, 9, 2]])

### Run on Test Data

In [28]:
test_bigger_risk_map = make_bigger_map(test_risk_map)
test_bigger_risk_graph = make_graph(test_bigger_risk_map)
calculate_risks(test_bigger_risk_graph, (0, 0))  # Should return 315

315

### Run on Input Data

In [29]:
bigger_risk_map = make_bigger_map(risk_map)
bigger_risk_graph = make_graph(bigger_risk_map)
calculate_risks(bigger_risk_graph, (0, 0))

3063